In [3]:
# %load ../env.py
%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline
#%load_ext rpy2.ipython

import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
import os 
from sklearn.metrics.pairwise import euclidean_distances,manhattan_distances,cosine_similarity

#Set environment variables


# Set up the local source files
#TOP = os.getcwd().replace('notebooks','')
TOP = "/share/home/ishah/ipynb/chiron/Camda18/"

LIB = TOP+'lib'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

os.environ['PYTHONPATH']=LIB


DAT_DIR = TOP + '/data/'
FIG_DIR = TOP + '/figs/'

CMAP_DIR= '/share/home/ishah/projects/HTTR/data/'


from db.mongo import *
#DB=openMongo(db='httr_ph1',host='pb.epa.gov')
#DB1=openMongo(db='httr_v1',host='pb.epa.gov')
MSG=openMongo(db='msigdb_v6',host='pb.epa.gov')
CMP=openMongo(db='cmap_v2',host='pb.epa.gov')
#from gexp.deseq2 import *
pd.options.display.max_colwidth = 500


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


# Goal

Match the cmap celfile identifiers to the chemicals

In [7]:
CMP.collection_names()

[u'mcf7_cmap_tgt_v1b',
 u'mcf7_v2a',
 u'chembl_activity',
 u'cmap1',
 u'mcf7_v2a_0',
 u'gene_corr',
 u'chembl_mech',
 u'genes1',
 u'mcf7_cmap_tgt_v1a',
 u'mcf7_wf_v2',
 u'cmap_mcf7_v1',
 u'gene_freq',
 u'cmap_tgt_new',
 u'mcf7_v1a',
 u'mcf7_v2b',
 u'chembl_mol',
 u'mcf7_v1b',
 u'mcf7_cmap_tgt_v2a',
 u'cmap_tgt_b',
 u'mcf7_wf_v1',
 u'mcf7_cmap_tgt_v2b',
 u'cmap_tgt_a',
 u'mcf7_v1a_ref']

## CAMDA

Load the provided file and annoations

In [33]:
CAMDA0 = pd.read_excel(DAT_DIR+'CAMDA_Challange_dataset_filenames.xlsx',skiprows=2)

In [86]:
D_mcf7 = CAMDA0.iloc[:,:5]
D_mcf7.columns=['s_id','data_type','tox','perturbation_scan_id','vehicle_scan_ids']
D_mcf7['cell']='MCF7'
D_pc3  = CAMDA0.iloc[:,[0,1,2,5,6]]
D_pc3.columns=['s_id','data_type','tox','perturbation_scan_id','vehicle_scan_ids']
D_pc3['cell']='PC3'

CAMDA1 = pd.concat((D_mcf7,D_pc3))

/opt/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Link with CMAP instances

In [35]:
CMap2 = pd.read_excel(DAT_DIR+'cmap_instances_02.xls')

In [37]:
def format_e(n):
    a = '%e' % n
    return a.split('e')[0].rstrip('0').rstrip('.') + 'e' + a.split('e')[1]

In [38]:
#CAMDA2 = CAMDA1.merge(CMap2[['perturbation_scan_id','instance_id','cmap_name','concentration (M)', u'duration (h)']],
#                      left_on='perturbation_scan_id',right_on='perturbation_scan_id')\
#        .rename(columns={'concentration (M)':'conc','duration (h)':'timeh'})
#Conc2=CAMDA2.conc.apply(format_e)
#CAMDA2['concs']=Conc2.astype(np.str)
#CAMDA2['camda_trt_id']=["%(cmap_name)s_%(cell)s_%(concs)s_%(timeh)i"%i for i in CAMDA2.to_dict('records')]

In [87]:
CAMDA1['perturbation_scan_id']=CAMDA2.perturbation_scan_id.str.replace("'","")

## Link with CMAP scans

In [75]:
CMap2_db=pd.DataFrame(list(CMP.cmap_trt_info.find(dict(cell={'$in':['PC3','MCF7']}),
                                          dict(_id=0,pert_id=1,name=1,conc=1,timeh=1))))\
         .drop_duplicates()
            

In [88]:
CAMDA2 = CAMDA1.merge(CMap2_db,left_on='perturbation_scan_id',right_on='pert_id',how='left')

In [1]:
CAMDA2.head()

NameError: name 'CAMDA2' is not defined